In [1]:
# Importing necessary library
import numpy as np
import matplotlib.pyplot as plt
import pywt
import pandas as pd
import os
import time
from scipy import signal
import sys
import warnings
import csv
warnings.filterwarnings('ignore')
%matplotlib inline

### Melakukan dekomposisi wavelet

In [2]:
def calcWaveletDec(data):
    # Select wavelet and decomposition level
    wavelet = 'db4'
    level = 5
    coeffs = pywt.wavedec(data, wavelet, level=level)
    return coeffs


### Menghitung energi

In [3]:
def calcEnergy(coeffs):
    # Calculate relative wavelet energy
    energies = []
    for c in coeffs:
        energies.append(np.sum(np.square(c)))

    decomp = ['A5', 'D1', 'D2', 'D3', 'D4', 'D5']
    energies[1:6] = energies[-1:-6:-1]

    total_energy = np.sum(energies)
    return energies

In [4]:
# Testing 

### Persiapan data

In [5]:
def get_csv_EEG(filename):
    # Load data from CSV
    df_data = pd.read_csv(filename)
    return df_data, df_data.columns


### Perhitungan RWB

In [6]:
def extract_feature(directory, destination, segment_time=1):
    fs = 256
    t = np.arange(0, 1, 1/(fs * segment_time))

    recap = pd.DataFrame(columns=['Wall Time', 'CPU Time'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            des_dir = os.path.join(destination +"_" , foldername).lower()
            files = os.listdir(folder)
            for filename in files:
                # print(f"Extract feature from {filename}...")
                cpu_start = time.process_time()
                wt_start = time.time()
                # if filename in zeros_train or filename in zeros_test:
                #     continue
                rel_path = os.path.join(directory, foldername, filename)
                # if 'metadata' in filename.lower():
                #     continue
                # trial_number = filename.split('.')[0].split('_')[1]
                df_data, channel_name = get_csv_EEG(rel_path)
                des_file = foldername +'_'+ filename + '_dwt' +'.npy'
                if not os.path.exists(des_dir):
                    os.makedirs(des_dir)
                des_path = os.path.join(des_dir, des_file)
                if os.path.exists(des_path):
                    continue
                DWT = []
                for channel in channel_name:
                    y = df_data[channel]; # sinyal per channel
                    # N = len(y);
                    # z = y - np.mean(y);
                    # nsamp = len (y[0])
                    energies = calcEnergy(calcWaveletDec(y))
                    for x in energies:
                        DWT.append(x)
                DWT = np.array(DWT)
                np.save(des_path, DWT)
                wt_end = time.time()
                cpu_end = time.process_time()
                wall_time = wt_end - wt_start
                cpu_time = cpu_end - cpu_start
                recap_temp = pd.DataFrame([[wall_time, cpu_time]],columns=recap.columns)
                recap = pd.concat([recap, recap_temp], ignore_index=True)
                # print(f"CPU Time: {cpu_time}, Wall Time: {wall_time}")
                # pd.DataFrame(DWT.T).to_csv(des_path, index=False)
    recap_dir = os.path.join('./logs/Execution',directory.split('/')[1])
    if not os.path.exists(recap_dir):
        os.makedirs(recap_dir)
    recap_path = os.path.join(recap_dir,'recap_dwt'+'.csv')
    recap.to_csv(recap_path)


### Mengecek nilai nol pada data

In [7]:
def check_zeros(directory):
    contain_zero = []
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                # if 'metadata' in filename.lower():
                #     continue
                df_data, channel_name = get_csv_EEG(rel_path)
                for channel in channel_name:
                    if (df_data[channel]== 0).all():
                        contain_zero.append(filename)
                        break
    return contain_zero

In [8]:
# check_zeros(os.path.join(directory_segmented, "autism"))

### Testing

In [9]:
def get_dummy(directory):
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            # des_dir = os.path.join(directory.replace('CSV', 'FEATURE')+"_" + str(lag),foldername).lower()
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                # if 'metadata' in filename.lower():
                #     continue
                # trial_number = filename.split('.')[0].split('_')[1]
                print(rel_path)
                df_data, channel_name = get_csv_EEG(rel_path)
                RWB = []
                return df_data[channel_name[0]]
                # pd.DataFrame(RWB.T).to_csv(des_path, index=False)
        


In [10]:
dummy = get_dummy("datasets/segmented_1 seconds/autism")

datasets/segmented_1 seconds/autism\Bader\segment_1.csv


In [11]:
coeffs = calcWaveletDec(dummy)

energies= calcEnergy(coeffs=coeffs)
print(energies)

[2405065.999507145, 363.8390952409968, 3038.5768248945, 6126.544840829188, 8506.21105168091, 45874.49335645977]


In [12]:
# print(relative_energies)

### Main Program

In [13]:
SEGMENT_TIME = 1

directory_segmented = f"datasets/segmented_{SEGMENT_TIME} seconds"
directory_feature = f"datasets/features/DWT/segment_{SEGMENT_TIME} seconds"
directory_logs = f"logs/Execution/segmented_{SEGMENT_TIME} seconds"

In [14]:
# # lags = [256, 128, 64, 32, 16, 8, 4, 2]
# lags = [128]

In [15]:
extract_feature(os.path.join(directory_segmented, "autism"), os.path.join(directory_feature, "autism"), segment_time=SEGMENT_TIME)
extract_feature(os.path.join(directory_segmented, "normal"), os.path.join(directory_feature, "normal"), segment_time=SEGMENT_TIME)

In [17]:
# lags = [256, 128, 64, 32, 16, 8, 4, 2]

SEGMENT_TIME = 1


df = pd.DataFrame()
for folder in os.listdir(directory_logs):
    temp = pd.read_csv(os.path.join(directory_logs, folder, f"recap_dwt.csv"))
    df = pd.concat([df, temp])
df = df.drop('Unnamed: 0', axis=1)   
print(df.sum(), "\n")

Wall Time    558.334913
CPU Time      28.937500
dtype: float64 

